<a href="https://colab.research.google.com/github/toot09/CF_WINGS_POS/blob/master/POS_CF_ONEPERSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source : https://data-science-hi.tistory.com/73

In [ ]:
import numpy as np
import pandas as pd
import math
import operator

In [ ]:
# 상품 마스터
items=[]
with open('POSITEM_061.csv','r',encoding='UTF-8') as f:
  for line in f.readlines():
    items.append(line.strip().split(","))
all_item = [i[0] for i in items[1:]]
print(len(all_item))
print(all_item[:2])

858
['신백설너비아니560G/냉동(씨제이제일제당)', '(A)치킨너겟700g(뉴)(씨제이제일제당)']


In [ ]:
# 매출 마스터
sales=[]
with open('2023_POS.csv','r',encoding='UTF-8') as f:
  for line in f.readlines():
    sales.append(line.strip().split(","))
print(sales[:2])

[['\ufeffregion', ' item', ' sale'], ['강원도', '간장치킨230G(하림)', '2']]


In [ ]:
# 지역별, 상품별 판매수량 Dict
def sales_dictionary(ratings):
  s_dict={}
  for i in sales[1:]:
    if i[0] not in s_dict:
      s_dict[i[0]] = {i[1]:float(i[2])}
    else:
      s_dict[i[0]].setdefault(i[1],float(i[2]))
  return s_dict

In [ ]:
# 코사인 유사도
def cosine_similarity(A,B):
  dot_p = np.dot(A,B)
  A_norms = math.sqrt(sum([i**2 for i in A]))
  B_norms = math.sqrt(sum([i**2 for i in B]))
  AB_norms = A_norms*B_norms
  return dot_p/AB_norms

In [ ]:
# 사용자기반 필터링 (User Based Filtering - UBF)
def user_based_filtering(sale_dict, region, k=3):
  # 지역의 매출 상품 집합(Set)
  region_s = set(sale_dict[region].keys())
  similar_score={}
  
  for other_region in sale_dict.keys():
    # region과 other_region의 공통 매출 상품 교집합(intersection)
    both_s = region_s.intersection(set(sale_dict[other_region].keys()))
    # 공통된 매출상품이 1개 이상인 경우만 고려
    if len(both_s)>=1:
      region_s_sale = [sale_dict[region][i] for i in both_s]
      region_other_sale = [sale_dict[other_region][i] for i in both_s]
      similar_score[other_region] = cosine_similarity(region_s_sale, region_other_sale)
  
  #유사도 높은 지역순으로 sort
  neighborhood = sorted(similar_score.items(), key=operator.itemgetter(1), reverse=True)
  print("■■■ 지역의 유사도 현황 ■■■")
  print(neighborhood[:50])

  # 추천을 위한 neighborhood 리스트
  region_for_recommendation = [i[0] for i in neighborhood]
  # region에 판매기록이 없는 상품리스트
  no_sale_i = set(all_item) - region_s

  # 추천결과
  recommendation_of_items = {}

  for item in no_sale_i:
    r=[]
    s=[]
    for region in region_for_recommendation:
      if item in sale_dict[region].keys():
        r.append(sale_dict[region][item]*similar_score[region])
        s.append(similar_score[region])
      else:
        continue
    # 판매상품기록이 없는 상품이 다른 지역에도 판매되지 않은경우
    if (len(r)==0):
      continue
    # k보다 작거나 같은 판매기록이 있는경우 사용자끼리의 가중평균을 구함
    elif len(r)<=k:
      recommendation_of_items[item] = sum(r)/sum(s)
    # K보다 많은 판매기록이 있는경우 정확도를 위해 상위 k번째의 지역에 대해서만 평균(가중평균)을 둠
    else:
      recommendation_of_items[item] = sum(r[:k])/sum(s[:k])
    
  return sorted(recommendation_of_items.items(), key=operator.itemgetter(1), reverse=True)[:1000]

In [ ]:
# 아이템 기반 필터링 (Item Based Filtering - IBF)
def item_based_filtering(sale_dict, region, k=3):
  # 임시 (region 별 매출이 있는 상품 list)
  #tmp = [list(sale_dict[i].keys()) for i in list(sale_dict.keys())]
  # 매출이 발생하지 않은 상품들 체크를 위한 list(데이터는 모든 매출이 발생한 상품)
  #no_sale_i = list(j for i in tmp for j in i)
  
  # 매출이 발생하지 않은 상품들 (ujone)
  no_sale_i = set(all_item) - sale_dict[region].keys()
  # Region에 매출 발생된 상품
  sale_i = [i for i in sale_dict[region].keys()]

  # 매출 발생되지 않은 상품에 대한 dict
  c_s = {}

  # j:매출발생된 상품 / i:매출발생하지 않은 상품(알아볼 상품)
  for i in no_sale_i:
    c_s[i] = {}
    for j in sale_i:
      p_i = []
      p_j = []
      for p in list(sale_dict.keys()):
        if j in sale_dict[p].keys() and i in sale_dict[p].keys():
          p_i.append(sale_dict[p][i])
          p_j.append(sale_dict[p][j])
      if len(p_i)>0:
        c = cosine_similarity(p_i, p_j)
        c_s[i].setdefault(j,c)
        #{ 매출발생하지 않은 상품 : {매출발생된 상품 : 코사인유사도} }
  
  print("■■■ 상품별 유사도 현황 ■■■")
  for i in list(c_s.keys())[:2]:
    tmp = sorted(c_s[i].items(), key=operator.itemgetter(1), reverse=True)
    print("예상 상품 : ",i," 매출발생 상품 : ",tmp[0]," 유사도 : ",tmp[1])
  
  # 결과
  result = {}

  # 매출 발생하지 않은 상품에 대한 코사인 유사도 계산
  for i in list(c_s.keys()):
    sum_sim_rating=[]
    sum_sim=[]
    # 코사인 유사도가 높은순으로 정렬
    k_i_j = sorted(c_s[i].items(), key=operator.itemgetter(1), reverse=True)

    for j in k_i_j:
      # 코사인 유사도
      sum_sim.append(j[1])
      # 코사인 유사도와 매출발생된 상품간의 예측계산
      sum_sim_rating.append( j[1]*sale_dict[region][j[0]] )

    if len(sum_sim)>0:
      # 매출 발생하지 않은 상품에 대한 예측 매출수량
      result[i] = sum(sum_sim_rating)/sum(sum_sim)

  return sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
# 지역별 판매현황
slDf = pd.DataFrame(np.array(sales[1:]),columns=sales[0])
# 지역별, 상품별 판매수량 { 지역 : {상품 : 판매수량} }
s_dict = sales_dictionary(slDf)

In [ ]:
# 사용자 기반 필터링
recommendation_1000 = user_based_filtering(s_dict, '제주특별자치도')
pd.DataFrame(np.array(recommendation_1000),columns=["상품명","예상판매수량"])


■■■ 지역의 유사도 현황 ■■■


,상품명,예상판매수량
0,명가등심돈까스(200G*5)1KG(아워홈),100.2612711497368
1,하림양념닭갈비(하림),85.0
2,빅핫도그140G(롯데푸드),68.30950689347937
3,동원미니돈까스450G(동원에프앤비),58.33503002556607
4,생가득올바른핫도그375G(풀무원),51.80590664478655
...,...,...
362,생가득현미취나물솥밥(2인분)420G(풀무원),1.0
363,치킨너겟550G(하림),1.0
364,모짜렐라핫도그400G(롯데푸드),1.0
365,하림)버팔로치킨봉(하림),1.0


In [ ]:
# 아이템 기반 필터링
recommendation_1000 = item_based_filtering(s_dict, '제주특별자치도')
pd.DataFrame(np.array(recommendation_1000),columns=["상품명","예상판매수량"])


■■■ 상품별 유사도 현황 ■■■
예상 상품 :  오징어볼튀김오징어53.0%1KG(한성기업)  매출발생 상품 :  ('감자튀김750G(롯데푸드)', 1.0)  유사도 :  ('고기동그랑땡800G(진주)', 1.0)
예상 상품 :  명가등심돈까스(200G*5)1KG(아워홈)  매출발생 상품 :  ('도톰등심돈까스1.5KG(한성기업)', 1.0)  유사도 :  ('명품동그랑땡1KG(동원에프앤비)', 1.0)


,상품명,예상판매수량
0,아워홈)숯불떡고기완자900g(아워홈),22.365853658536587
1,A)프레시안/빕스새우볶음밥200G/냉동(씨제이제일제당),22.365853658536587
2,사천풍중화볶음밥450G(롯데푸드),20.32075471698113
3,하림양념닭갈비(하림),19.55952380952381
4,청정원고메레시피차슈볶음밥양념(13G*2)26G(대상),19.55952380952381
...,...,...
362,대림선잡채해물완자1KG(사조),13.905467112187186
363,오쉐프새우볶음밥300G(오뚜기),13.765914722140515
364,대림선동그랑땡1KG(사조),13.487458389408687
365,하림)닭다리후라이드1Kg(하림),13.352426398709063
